# MiniProejct

주제: 문화유적 해설사  

소크코드 설명 : PDF 문서를 RAG해서 챗봇을 만들 데이터를 저장합니다

### 환경설정
API_KEY를 설정합니다

In [94]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [ ]:
#import os
# API key확인
#print(f"[API KEY]\n{os.environ['LLAMA_CLOUD_API_KEY'][:5]}" + "*" * 15)

LangSmith를 설정합니다

In [95]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("RAG-PRJ")

LangSmith 추적을 시작합니다.
[프로젝트명]
RAG-PRJ-MD


필요한 라이브러리를 import 합니다

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from llama_parse import LlamaParse
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
import os

## 1~8단계까지 순서대로 구현해보기
### 단계 1: 문서 로드(Load Documents)
LlamaParse로 PDF 문서 내용을 불러옵니다

In [ ]:
# documents = LlamaParse(result_type="markdown")

# 동기화 오류때문에 추가
import nest_asyncio

nest_asyncio.apply()

def parse_file_to_markdown_pages(file_path):
    """다양한 파일 형식을 페이지별 Markdown으로 변환"""
    try:
        # 기본 prompt (PDF, 이미지, 기타 등)
        system_prompt = (
            "You are parsing a document. Please extract all content including tables in markdown format. "
            "Preserve the structure and formatting as much as possible. "
            "Mark page breaks clearly with '---PAGE_BREAK---' marker."
        )
        # LlamaParse 설정
        parser = LlamaParse(
            use_vendor_multimodal_model=True,
            vendor_multimodal_model_name="openai-gpt4o",
            vendor_multimodal_api_key=os.environ.get("OPENAI_API_KEY"),
            result_type="markdown",
            language="ko",
            system_prompt=system_prompt,
        )

        # 파일 파싱
        parsed_docs = parser.load_data(file_path=file_path)

        return parsed_docs

    except Exception as e:
        print(f"오류: {str(e)}")
        return []
    

documents = parse_file_to_markdown_pages(file_path = "./data/deoksugung.pdf")

Started parsing the file under job_id 443d1957-c326-468f-9a45-94c4a3aa6316


In [7]:
len(documents)

18

In [8]:
# langchain 도큐먼트로 변환
docs = [doc.to_langchain_format() for doc in documents]

In [28]:
print(docs[3].page_content)




# 1 대한문 일원
大漢門

----

**대한제국, 근대 세계를 향해 문을 열다**

원래 경운궁의 정문은 남쪽으로 난 인화문(仁化門)이었다. 그런데 동문(東門)이었던 대한문(大漢門) 앞으로 여러 방향의 도로들이 건설되고 환구단이 건립되면서 궁궐의 동쪽이 새로운 도심이 되자 대한문의 실질적인 정문 역할을 하게 되었다. 대한문은 '크게 편안하다'는 뜻인데 1906년 수리와 함께 대환문(大漢門)으로 이름이 바뀌었다. '대한(大漢)'은 '한양이 창대해진다'는 뜻이다. 《대한문상량문(大漢門上樑文)》을 보면 "황하가 맑아지는 천제일유의 시운을 맞았으므로 국운이 길이 창대할 것이고 한양이 억만 년 이어갈 터전에 자리하였으니 문 이름으로 특별히 건다."고 나와 있다. 한양을 수도로 하여 새로 태어난 대한제국의 영원히 창대하라는 염원을 담은 말이다. 대한문을 지나 건너게 되는 금천교는 1986년 발굴되어 정비한 것이다. 이 다리를 건너 중화문 앞에 이르는 길이 궁궐의 중심 행차로였다.

!대한문





### 단계 2: 문서 분할(Split Documents)


In [10]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)
print(f"분할된 청크의수: {len(split_documents)}")

분할된 청크의수: 37


In [11]:
split_documents

[Document(metadata={}, page_content='관람정보\n\n# 관람정보\n\n100-120 서울시 중구 세종대로 99 / deoksugung.go.kr / 02-771-9952\n\n## 관람시간\n09:00~21:00  \n※ 매일 1시간 전까지 입장해야 하며 매주 월요일은 쉽니다\n\n## 입장요금\n- 어른(19~64세) 1,000원(10인 이상 800원)\n\n## 무료 안내 시간\n\n### 국어\n- 화~금요일: 10:00, 11:00, 14:00, 15:00, 16:00\n- 토요일: 10:00, 11:00, 14:00, 15:00, 16:30\n- 일요일: 10:00, 11:00, 14:00, 15:00, 16:30\n\n### 영어\n- 화, 수, 금요일: 10:30\n- 토, 일요일: 13:40\n\n### 일어\n- 화, 수, 금요일: 10:30\n- 화, 수, 금요일: 14:30\n- 토요일: 10:30, 14:30\n- 일요일: 10:30\n\n### 중국어\n- 화, 수, 금요일: 13:40\n- 토요일: 13:40\n- 일요일: 13:40'),
 Document(metadata={}, page_content='- 궁궐코스 내 안내판 앞에서 출발하며 약 1시간 정도 소요됩니다\n- 공휴일에는 쉬지 않습니다\n- 사전예약시 전액 무료 해설\n- 우리궁궐지킴이(02-723-4206, www.rekor.or.kr)\n- 우리궁궐관리소(02-2273-2276, www.palaceguide.or.kr)\n\n## 교통안내\n\n- 지하철: 시청역(1호선) 연결, 을지로입구역(2호선) 12번 출구\n- 버스: \n  - 간선(파랑): 101, 150, 172, 263, 401, 402, 406, 408, 421, 472, 504, 507, 600, 602, 603, 604, 700, 703\n  - 지선(초록): 601, 7012, 7015, 7016, 7019, 7022\n  - 공항(빨강): 8600\n- 자세한 사항은 

### 단계 3: 임베딩(Embedding) 생성

In [12]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

### 단계 4: DB 생성(Create DB) 및 저장

FAISS를 활용합니다

In [13]:
# 벡터스토어에 임베딩과정을 거쳐서 저장
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

In [14]:
# 검색으로 임베딩 결과를 확인
for doc in vectorstore.similarity_search("석조전"):
    print(doc.page_content)
    print("==="*30)

# 6 석조전 일원
石造殿

----

## 근대국가의 상징

석조전은 고종황제가 침전 겸 편전으로 사용하기 위해 1900년부터 1910년에 걸쳐 지은 서양식 석조건물이다. 경운궁에 서양식 건축물들을 건립한 것은 대한제국 근대화를 위한 정책의 일환이었다. 석조전은 서양의 신고전주의 건축양식으로 지어졌으며, 건물의 앞과 동서 양편에 베란다가 설치된 것이 특징이다. 1층은 시종이 기거하는 방과 부속 시설로, 돌계단을 올라 들어서는 2층은 대접견실과 대기실로, 3층은 황제와 황후가 거처하는 침실과 여러 용도의 방으로 구성되었다. 고종황제 승하 후에 경운궁이 황폐해지는 과정에서 석조전은 일본 회화미술관으로 사용되었다. 1938년에 서측 별관이 들어서 이왕가미술관으로 사용되기도 하였다.
# 석조전을 시공한 오쿠라 토목회사

석조전은 처음에 대한제국의 재정고문이었던 영국인 브라운(J. M. Brown)이 발의하여 짓기 시작했으며 설계도 영국인 하딩(G. R. Harding) 등이 하였다. 그러다가 1905년에 일본인 메가타 대체제국의 재정고문이 되면서 일본의 오쿠라 토목회사가 시공을 맡게 된다. 오쿠라 토목회사는 아시아 여러 나라에 수많은 건축 화재를 일본으로 빼돌린 기업으로도 유명하다. 실제로 1917년 경복궁 자선당을 해체해 일본으로 빼돌려 오쿠라의 개인 저택에 ‘아고 조선관’이라는 이름의 사설박물관으로 사용했다. 석조전은 1923년 관동 대지진으로 불타 버려 초석만 남아 있다가 1995년 한국으로 돌아왔다.

!Image of a historical artifact

11
# 석조전 앞의 분수

석조전 앞에 있는 정원과 청동제 분수는 서구형 정원으로 1938년에 조성된 것으로 알려져 있다. 우리의 전통 정원은 건물의 뒤에 배치하여 후원이라 하는 데 비해, 석조전은 건물 앞에 정원을 조성하였다. 또한 우리 전통 조경에서는 분수를 찾아볼 수 없는데, 이는 한국인의 자연관에서 비롯된 것이다. 우리 조상들은 물을 높은 곳에서 낮은 곳으로 흐르는 것이 자연의 이치라 여겼기에

In [15]:
# 로컬 Disk 에 저장
vectorstore.save_local(folder_path="faiss_db", index_name="faiss_index")

### 단계 5: 검색기(Retriever) 생성

In [16]:
# 문서내용에서 정보를 검색하고 답을 생성
retriever = vectorstore.as_retriever()

In [17]:
# 검색기에 쿼리를 날려 검색된 chunk 결과를 확인
retriever.invoke("석조전에 용도가 뭐야?")

[Document(id='3c126a05-f6d6-4568-b20b-03b7682b696d', metadata={}, page_content='# 6 석조전 일원\n石造殿\n\n----\n\n## 근대국가의 상징\n\n석조전은 고종황제가 침전 겸 편전으로 사용하기 위해 1900년부터 1910년에 걸쳐 지은 서양식 석조건물이다. 경운궁에 서양식 건축물들을 건립한 것은 대한제국 근대화를 위한 정책의 일환이었다. 석조전은 서양의 신고전주의 건축양식으로 지어졌으며, 건물의 앞과 동서 양편에 베란다가 설치된 것이 특징이다. 1층은 시종이 기거하는 방과 부속 시설로, 돌계단을 올라 들어서는 2층은 대접견실과 대기실로, 3층은 황제와 황후가 거처하는 침실과 여러 용도의 방으로 구성되었다. 고종황제 승하 후에 경운궁이 황폐해지는 과정에서 석조전은 일본 회화미술관으로 사용되었다. 1938년에 서측 별관이 들어서 이왕가미술관으로 사용되기도 하였다.'),
 Document(id='9c3cdeac-5756-4173-9392-d171ad49c3b5', metadata={}, page_content='# 석조전 앞의 분수\n\n석조전 앞에 있는 정원과 청동제 분수는 서구형 정원으로 1938년에 조성된 것으로 알려져 있다. 우리의 전통 정원은 건물의 뒤에 배치하여 후원이라 하는 데 비해, 석조전은 건물 앞에 정원을 조성하였다. 또한 우리 전통 조경에서는 분수를 찾아볼 수 없는데, 이는 한국인의 자연관에서 비롯된 것이다. 우리 조상들은 물을 높은 곳에서 낮은 곳으로 흐르는 것이 자연의 이치라 여겼기에, 정원에 인공으로 폭포를 만들긴 했지만 물이 억으로 치솟는 분수는 만들지 않았던 것이다.\n\n!Image of a fountain in front of Seokjojeon\n\n# 덕수궁 미술관\n\n덕수궁 미술관 圖書館\n\n광명문 光明門\n\n석조전 石造殿\n\n!Illustration of Deoksugung Palace\n\n# 석조전을 시공한 오쿠라 토목회사'),
 Document

### 단계 6: 프롬프트 생성(Create Prompt)

In [18]:
# 프롬프트를 생성
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer in Korean.

#Context: 
{context}

#Question:
{question}

#Answer:"""
)

### 단계 7: 언어모델(LLM) 생성

In [19]:

# 모델(LLM) 을 생성합니다.
llm = ChatOpenAI(model_name="gpt-4.1-mini", temperature=0)

### 단계 8: 체인(Chain) 생성

In [20]:

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser() # 출력에 page_content내용만 보일 수 있도록
)

In [21]:
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "석조전의 용도는 무엇인가요?"
response = chain.invoke(question)
print(response)

석조전은 고종황제가 침전 겸 편전으로 사용하기 위해 지은 서양식 건물입니다. 1층은 시종이 기거하는 방과 부속 시설, 2층은 대접견실과 대기실, 3층은 황제와 황후가 거처하는 침실과 여러 용도의 방으로 구성되어 있습니다. 즉, 석조전은 황제의 거처이자 공식적인 접견 장소로 사용되었습니다.


In [22]:
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "덕수궁에 돌로만든 건물의 이름은 뭐지?"
response = chain.invoke(question)
print(response)

덕수궁에 돌로 만든 건물의 이름은 석조전입니다.


## FAISS

In [23]:
# 저장된 데이터를 로드
db = FAISS.load_local(
    folder_path="faiss_db",
    index_name="faiss_index",
    embeddings=embeddings,
    allow_dangerous_deserialization=True,
)

In [24]:
# db 의 데이터 확인
db.index_to_docstore_id

{0: '4a476652-4267-45ce-b7f4-ffa22634a2fc',
 1: '3ad2aa8f-8565-456d-9229-ebcd8f6d0a32',
 2: '6c139653-026c-4514-aa30-21a186ccc042',
 3: '125ed819-5b33-4685-8df9-0d2f99ad1cf0',
 4: '775031b2-7a51-4881-8f41-dcda465d5350',
 5: 'eeef7646-8e15-44a1-9015-251654f355b1',
 6: 'a352a794-83f5-41cf-a64d-567a8ffa821a',
 7: '3992fa66-4b81-488f-ac1d-a11256d49a7d',
 8: 'f7f2f3fd-24a4-4295-82b4-bc9b979938cb',
 9: '0426f607-5241-4a62-abb6-923cef454e23',
 10: 'a05107bf-38c5-4297-a6df-71e8eb17b9f6',
 11: '831de023-6ac3-4435-a63e-dbf448c73c81',
 12: 'c21b105d-46f1-404d-a44c-029f149cd948',
 13: '8e617aae-2874-47ca-aadf-4b52699f09ae',
 14: '03c84490-6fc8-488c-ac04-0a565fed985f',
 15: 'eb7d35aa-72af-4149-8cd8-94f955b3214a',
 16: '7ce56ae6-f9c9-4af5-892f-447292aaf8e7',
 17: '4f765aac-2694-4bf1-a0fa-ba0770631830',
 18: 'fe005cd2-73bd-4d20-b557-902ba9f3aa33',
 19: 'fb460b2d-d75f-4232-8e56-00fc5aa0f6b6',
 20: 'f1d0292b-a4d7-4e7e-a826-3016e27a164f',
 21: '9d5a7297-4f99-4201-a891-2b6d4d48cafa',
 22: 'add56cc9-03ef-

## markdown파일을 변형해서 검색성능 비교

1. PDF 문서를 로드해서 markdown으로 저장합니다  
2. 저장된 makedown문서를 \n\n\n으로 구분되도록 편집합니다  
3. 편집하지 않고 임베딩을 해서 벡터스토어에 저장한 것과 편집해서 임베딩한 문서의 검색성능을 비교합니다  

md파일경로 : C:\Users\yujap\github\langchain-kr\88_myproject\data\deoksugung.md

In [ ]:
# 문서를 로드하고 markdown 파일로 저장
import os
from llama_parse import LlamaParse
# 동기화 오류때문에 추가
import nest_asyncio

nest_asyncio.apply()

documents = LlamaParse(result_type="markdown")


def pdf_parser(pdf_file_path: str):
    """
    PDF 파일을 파싱하여 그 내용을 Markdown 파일로 저장합니다.

    Args:
        pdf_file_path (str): 처리할 PDF 파일의 경로.
    """
    print(f"🔄 '{pdf_file_path}' 파일 파싱을 시작합니다...")

    try:
        # parsing instruction 을 지정합니다.
        parsing_instruction = (
            "You are parsing a AI Report. Please extract tables in markdown format."
        )

        # LlamaParse 설정
        parser = LlamaParse(
            use_vendor_multimodal_model=True,
            vendor_multimodal_model_name="openai-gpt4o", # 모델설정
            vendor_multimodal_api_key=os.environ["OPENAI_API_KEY"],
            result_type="markdown", # 결과 출력 형식
            # high_res_ocr=True,
            parsing_mode="Unstructured",
            language="ko", # 언어
            parsing_instruction=parsing_instruction, 
        )

        # 1. LlamaParse를 사용하여 PDF 파일을 로드합니다.
        # 'documents' 객체는 이 함수 외부에서 미리 정의되어 있어야 합니다.
        parsed_docs = documents.load_data(file_path=pdf_file_path)

        # 2. LangChain 형식의 도큐먼트로 변환합니다.
        docs = [doc.to_langchain_format() for doc in parsed_docs]

        # 3. 저장할 Markdown 파일의 경로를 생성합니다. (확장자 변경)
        file_root, _ = os.path.splitext(pdf_file_path)
        output_file_path = file_root + ".md"

        # 4. 모든 페이지의 내용을 하나의 텍스트로 합칩니다.
        #    페이지 사이는 두 줄로 띄어 가독성을 높입니다.
        full_text = "\n\n".join([doc.page_content for doc in docs])

        # 5. 추출된 전체 텍스트를 .md 파일로 저장합니다.
        with open(output_file_path, "w", encoding="utf-8") as f:
            f.write(full_text)

        print(f"✅ 파일 저장 완료: {output_file_path}")

    except FileNotFoundError:
        print(f"❌ 오류: 파일을 찾을 수 없습니다 - {pdf_file_path}")
    except Exception as e:
        print(f"❌ 오류 발생: {e}")


# --- 함수 사용 예시 ---
# 이 코드를 실행하기 전에 'documents' 파서 객체를 초기화해야 합니다.
# file_to_parse = "data/디지털정부혁신추진계획.pdf"
file_to_parse = "./data/deoksugung.pdf"
pdf_parser(file_to_parse)

🔄 './data/deoksugung.pdf' 파일 파싱을 시작합니다...
Started parsing the file under job_id e3c7ee3f-d6cc-4178-bd69-508a6fe74a6e
✅ 파일 저장 완료: ./data/deoksugung.md


In [78]:
from IPython.display import display, Markdown
from langchain.schema import Document

def display_md_file_in_notebook(file_path):
    """
    MD 파일을 읽어와 Jupyter Notebook에 마크다운 형식으로 표시합니다.
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            md_content = f.read()
            # display(Markdown(md_content))
        return md_content
    except FileNotFoundError:
        print(f"오류: '{file_path}' 파일을 찾을 수 없습니다.")
    except Exception as e:
        print(f"파일을 읽는 중 오류가 발생했습니다: {e}")
        return []

# 사용 예시
# './data/deoksugung.md' 파일을 로드하여 표시합니다.
file_to_display = './data/deoksugung.md'
markdown_text = display_md_file_in_notebook(file_to_display)

In [79]:
def print_doc_contecnt(docs):
    for i,con in enumerate(docs):
        print(f"{i+1}번째 문서")
        print(f"내용:\n{con.page_content}")
        print("---" * 30)

In [80]:
markdown_textdocs = [Document(page_content=markdown_text, metadata={"source": file_to_display})]

In [ ]:
markdown_textdocs

In [86]:
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n \n \n","\n\n"," "],
    chunk_size=600, 
    chunk_overlap=50,
    )
split_documents = text_splitter.split_documents(markdown_textdocs)
print(f"분할된 청크의수: {len(split_documents)}")

분할된 청크의수: 28


In [87]:
split_documents[0].page_content
#split_documents

'# 관람정보\n100-120 서울시 중구 세종대로 99 / deoksugung.go.kr / 02-771-9952\n  \n  \n  \n## 관람시간\n09:00~21:00  \n※ 매일 1시간 전까지 입장해야 하며 매주 월요일은 쉽니다\n  \n  \n  \n## 입장요금\n- 어른(19~64세) 1,000원(10인 이상 800원)\n  \n  \n  \n## 무료 안내 시간\n\n### 국어\n- 화~금요일: 10:00, 11:00, 14:00, 15:00, 16:00\n- 토요일: 10:00, 11:00, 14:00, 15:00, 16:30\n- 일요일: 10:00, 11:00, 14:00, 15:00, 16:30\n\n### 영어\n- 화, 수, 금요일: 10:30\n- 토, 일요일: 13:40\n\n### 일어\n- 화, 수, 금요일: 10:30\n- 화, 수, 금요일: 14:30\n- 토요일: 10:30, 14:30\n- 일요일: 10:30\n\n### 중국어\n- 화, 수, 금요일: 13:40\n- 토요일: 13:40\n- 일요일: 13:40'

In [88]:
# 단계 3: 임베딩(Embedding) 생성
# 단계 4: DB 생성(Create DB) 및 저장
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

In [89]:
# 로컬 Disk 에 저장
vectorstore.save_local(folder_path="faiss_db2", index_name="faiss_index")

In [90]:
for doc in vectorstore.similarity_search("인목왕후"):
    print(doc.page_content)
    print("==="*30)

석어당은 선조의 계비인 인목왕후 김씨가 10여 년간 감금생활을 했던 곳이다. 광해군은 왕위에 오른 후 자신의 왕위를 위협할 가능성이 있는 형제들을 차례로 제거하고 인목대비를 폐위시켜 경운궁에 유폐했다. 이때 경운궁은 서궁(西宮)으로 불렸다. 서궁 유폐는 결국 반정을 일으키는 구실이 되었다. 반정이 성공한 능양군(인조)은 경운궁으로 입궐해를 찾아 정통성을 인정받고 여기서 즉위했다. 유폐의 한이 맺혀 있던 인목대비는 석어당 앞마당에 광해군을 끌어내리고 36조의 죄를 물은 후 능운궁에서 묘사를 전하였다.
  
  
  
4 함녕전과 덕홍전

# 함녕전과 덕홍전
### 咸寧殿 · 德弘殿

### 황제의 침전과 접견실
1897년에는 고종의 침전과 정부 각 부처를 연결하는 전화가 설치되었다. 함녕전의 대청마루에 전화기가 설치되자, 고종은 필요할 때마다 전화기로 대신들에게 지시를 내렸다. 이 전화를 '대청 전화'라고 했는데 대전쟁적 덕분에 김구가 사형 직전에 목숨을 건진 일화가 '백범일지'에 실려 있다. 고종이 김구의 사형집행 서류를 검토 중 함경령 직전에 전화로 사형집행정지 명령을 내려 목숨을 구했다는 것이다.

!Image of 함녕전
  
  
  
5 정관헌 靜觀軒

# 전통 건축과 서양 건축의 조화
!Image of 중명전 일원
  
## 중명전에서 공포 분위기 속에 을사늑약 체결

1905년 11월 17일 밤, 일제는 경운궁 안팎에 무장한 일본군을 배치하고 당시 고종이 기거하고 있던 중명전에서 고종과 대신들에게 보호조약에 조인할 것을 강요하였다. 이날 일본공사가 하야시가 이미 일본대사관으로 대사를 불러 협박해 체결을 종용하였으나 오후가 되도록 뜻을 이루지 못하자 경운궁에서 전화회를 열어 강요하였다. 공포 분위기 속에서도 고종과 대신들은 여전히 조인을 거부하였고, 이를 해결하는 세 번에 걸쳐 고종을 협박하였다. 고종이 끝내 거부하자 이토는 찬성하는 대신들(을사오적)만 데리고 조약을 체결하였다. 고종은 서명이나 옥새 날인을 하지 않음으로써 끝까지 이를 승인하지 않았

In [91]:
retriever = vectorstore.as_retriever()
query = "인목왕후가 감금되어 있었던 곳인 어디인가?"

# 리트리버를 사용하여 검색
retrieved_docs = retriever.invoke(query)
print_doc_contecnt(retrieved_docs)

1번째 문서
내용:
석어당은 선조의 계비인 인목왕후 김씨가 10여 년간 감금생활을 했던 곳이다. 광해군은 왕위에 오른 후 자신의 왕위를 위협할 가능성이 있는 형제들을 차례로 제거하고 인목대비를 폐위시켜 경운궁에 유폐했다. 이때 경운궁은 서궁(西宮)으로 불렸다. 서궁 유폐는 결국 반정을 일으키는 구실이 되었다. 반정이 성공한 능양군(인조)은 경운궁으로 입궐해를 찾아 정통성을 인정받고 여기서 즉위했다. 유폐의 한이 맺혀 있던 인목대비는 석어당 앞마당에 광해군을 끌어내리고 36조의 죄를 물은 후 능운궁에서 묘사를 전하였다.
  
  
  
4 함녕전과 덕홍전

# 함녕전과 덕홍전
### 咸寧殿 · 德弘殿

### 황제의 침전과 접견실
------------------------------------------------------------------------------------------
2번째 문서
내용:
## 환구단, 황제국 선포를 하늘에 고하다

현재의 덕수궁 동쪽 담장 안쪽과 담장 너머 서울광장 일부는 궁궐 안 관청들이 있던 권내각사 일원이었다. 경운궁으로 정궁을 이전하면서 군사권을 관장하던 원수부와 황실의 업무를 보던 궁내부를 비롯해 기양원, 태의원, 전화국 등 여러 관청이 권내에 위치하게 되었다. 후에 태평로 개설로 인해 절반 이상의 건각이 사라졌고, 나머지 건각 역시 1933년 공원으로 조성되는 과정에서 철거되었다. 환구단은 현 서울광장 너머에 있었던 남별궁 터(현 조선호텔 자리)에 건축되었다. 남별궁은 본디 태종의 둘째 공주인 경정공주가 거주했던 곳으로 1583년(선조 16년)에 의안군이 거주하면서 남별궁이라 불렸다. 이후 인조 때 중국 사신을 접대하던 태평관이 철폐되자 중국 사신의 거처로 사용되다가, 1897년 고종이 황제에 오름을 하늘에 고하기 위한 환구단이 이곳에 건축되었다. 이후 1914년 조선철도호텔 건설 때 파괴되었으나 신위를 모셨던 황궁우와 돌북(石鼓)이 남아 대한제국의 흔적을 보여주고 있다. 환구단은 사적 제157호로 지정되었다.


In [40]:
query = "고종이 침전으로 사용했던 건물이 뭐야?"
# 리트리버를 사용하여 검색
retrieved_docs = retriever.invoke(query)
print_doc_contecnt(retrieved_docs)

1번째 문서
내용:
## 근대국가의 상징

석조전은 고종황제가 침전 겸 편전으로 사용하기 위해 1900년부터 1910년에 걸쳐 지은 서양식 석조건물이다. 경운궁에 서양식 건축물들을 건립한 것은 대한제국 근대화를 위한 정책의 일환이었다. 석조전은 서양의 신고전주의 건축양식으로 지어졌으며, 건물의 앞과 동서 양편에 베란다가 설치된 것이 특징이다. 1층은 시종이 기거하는 방과 부속 시설로, 돌계단을 올라 들어서는 2층은 대접견실과 대기실로, 3층은 황제와 황후가 거처하는 침실과 여러 용도의 방으로 구성되었다. 고종황제 승하 후에 경운궁이 황폐해지는 과정에서 석조전은 일본 회화미술관으로 사용되었다. 1938년에 서측 별관이 들어서 이왕가미술관으로 사용되기도 하였다.
  
  
  
# 석조전 앞의 분수
------------------------------------------------------------------------------------------
2번째 문서
내용:
# 전통 건축과 서양 건축의 조화

정관헌(靜觀軒)은 그 이름처럼 궁궐 후원의 언덕 위에서 ‘조용히 궁궐을 내려다보는’ 휴식용 건물이다. 위치도 함녕전 뒤에 자리 잡고 있어, 전통 궁궐에서 후원의 정자 기능을 대신하는 건물이라 할 수 있다. 1900년경 러시아 건축가 사바틴(A. I. Sabatin)이 한식과 양식을 절충해 설계한 건축물이다. 기단 위에 로마네스크 양식의 인조석 기둥을 둘러서 내부 공간을 만들었고, 바깥에는 동·남·서 세 방향에 기둥을 세운 베란다가 둘러져 있다. 흥미로운 것은 석재를 기본으로 하는 서양식 기둥이 나무로 만들어졌다는 점과, 기둥 상부에 청룡과 황룡, 박쥐, 꽃병 등 한국 전통 문양이 새겨져 있다는 사실이다. 고종황제는 정관헌에서 커피를 마시며 외교 사절들과 연회를 즐겼다고 한다.

!정관헌 이미지
  
  
  
## 커피와 고종 독살미수 사건
----------------------------------------------------------------

In [41]:

query = "석조전에 용도가 뭐야?"
retrieved_docs = retriever.invoke(query)
print_doc_contecnt(retrieved_docs)

1번째 문서
내용:
## 근대국가의 상징

석조전은 고종황제가 침전 겸 편전으로 사용하기 위해 1900년부터 1910년에 걸쳐 지은 서양식 석조건물이다. 경운궁에 서양식 건축물들을 건립한 것은 대한제국 근대화를 위한 정책의 일환이었다. 석조전은 서양의 신고전주의 건축양식으로 지어졌으며, 건물의 앞과 동서 양편에 베란다가 설치된 것이 특징이다. 1층은 시종이 기거하는 방과 부속 시설로, 돌계단을 올라 들어서는 2층은 대접견실과 대기실로, 3층은 황제와 황후가 거처하는 침실과 여러 용도의 방으로 구성되었다. 고종황제 승하 후에 경운궁이 황폐해지는 과정에서 석조전은 일본 회화미술관으로 사용되었다. 1938년에 서측 별관이 들어서 이왕가미술관으로 사용되기도 하였다.
  
  
  
# 석조전 앞의 분수
------------------------------------------------------------------------------------------
2번째 문서
내용:
석어당은 선조의 계비인 인목왕후 김씨가 10여 년간 감금생활을 했던 곳이다. 광해군은 왕위에 오른 후 자신의 왕위를 위협할 가능성이 있는 형제들을 차례로 제거하고 인목대비를 폐위시켜 경운궁에 유폐했다. 이때 경운궁은 서궁(西宮)으로 불렸다. 서궁 유폐는 결국 반정을 일으키는 구실이 되었다. 반정이 성공한 능양군(인조)은 경운궁으로 입궐해를 찾아 정통성을 인정받고 여기서 즉위했다. 유폐의 한이 맺혀 있던 인목대비는 석어당 앞마당에 광해군을 끌어내리고 36조의 죄를 물은 후 능운궁에서 묘사를 전하였다.
  
  
  
4 함녕전과 덕홍전

# 함녕전과 덕홍전
### 咸寧殿 · 德弘殿

### 황제의 침전과 접견실
------------------------------------------------------------------------------------------
3번째 문서
내용:
석조전은 처음에 대한제국의 재정고문이었던 영국인 브라운(J. M. Brown)이 발의하여 짓기 시작했으

In [42]:
query = "이용시간을 알려줘"
retrieved_docs = retriever.invoke(query)
print_doc_contecnt(retrieved_docs)

1번째 문서
내용:
# 관람정보
100-120 서울시 중구 세종대로 99 / deoksugung.go.kr / 02-771-9952
  
  
  
## 관람시간
09:00~21:00  
※ 매일 1시간 전까지 입장해야 하며 매주 월요일은 쉽니다
  
  
  
## 입장요금
- 어른(19~64세) 1,000원(10인 이상 800원)
  
  
  
## 무료 안내 시간

### 국어
- 화~금요일: 10:00, 11:00, 14:00, 15:00, 16:00
- 토요일: 10:00, 11:00, 14:00, 15:00, 16:30
- 일요일: 10:00, 11:00, 14:00, 15:00, 16:30

### 영어
- 화, 수, 금요일: 10:30
- 토, 일요일: 13:40

### 일어
- 화, 수, 금요일: 10:30
- 화, 수, 금요일: 14:30
- 토요일: 10:30, 14:30
- 일요일: 10:30

### 중국어
- 화, 수, 금요일: 13:40
- 토요일: 13:40
- 일요일: 13:40
------------------------------------------------------------------------------------------
2번째 문서
내용:
- 궁궐코스 내 안내판 앞에서 출발하며 약 1시간 정도 소요됩니다
- 공휴일에는 쉬지 않습니다
- 사전예약시 전액 무료 해설
- 우리궁궐지킴이(02-723-4206, www.rekor.or.kr)
- 우리궁궐관리소(02-2273-2276, www.palaceguide.or.kr)
  
  
  
## 교통안내

- 지하철: 시청역(1호선) 연결, 을지로입구역(2호선) 12번 출구
- 버스: 
  - 간선(파랑): 101, 150, 172, 263, 401, 402, 406, 408, 421, 472, 504, 507, 600, 602, 603, 604, 700, 703
  - 지선(초록): 601, 7012, 7015, 7016, 7019, 7022
  - 공항(빨강): 

In [43]:
query = "덕수궁에 대해 알려줘"
retrieved_docs = retriever.invoke(query)
print_doc_contecnt(retrieved_docs)

1번째 문서
내용:
# 덕수궁

- 1593 (선조 26) 월산대군 저택을 행궁(행궁)으로 사용
- 1611 (광해 3) '경운궁'이라는 궁호를 붙임
- 1615 (광해 7) 경운궁이 창덕궁으로 이어짐에 변경이 됨
- 1618 (광해 10) 인목대비 투옥, 서궁(西宮)으로 불림
- 1623 (인조 1) 광운궁 대부인과 가덕대부인에게 물려줌
- 1897 (고종 24, 25) 러시아공사관에서 경운궁으로 옮겨 옴
- 1897 (광무 1, 2) 한국에서 황제즉위식 거행
- 1904 (광무 8) 대화재로 주요 건물들이 대부분 소실
- 1907 (융희 1) 순종 황제 대관식 내정
- 1919 고종, 덕수궁 함녕전에서 승하
- 1933 덕수궁, 일본인에게 개방
- 1938 덕수궁 석조전 옆에 분수대 설치
- 2007 덕수궁 복원 시작
  
  
     
## 정보

1. 표 파는 곳  
2. 지하철  
3. 화장실  
4. 기념품점  
5. 휠체어 보관소

## 권역
------------------------------------------------------------------------------------------
2번째 문서
내용:
경운궁은 1897년 대한제국 출범과 함께 한국 근대사의 전면에 등장했다. 전성기 때의 경운궁은 현재 남아 있는 3배에 달하는 큰 궁궐이었다. 현재의 미국대사관 건너편 저쪽에는 중명전을 비롯해 황실 생활을 위한 전각들이 있었고, 북쪽에는 역대 임금들을 제사 지내는 선원전 일원이 있었으며, 동쪽에는 하늘에 제사 지내는 원구단은 실질하여 황제국의 위세를 과시했다. 그러나 고종황제가 황해에 물러나면서 경운궁은 선원제가 거처하는 궁으로 그 위상이 달라졌고 이름도 덕수궁으로 바뀌었다. 이후 칸나자 일부와 환구단이 철거되었고 이 지역은 대한제국의 상징 공간으로서의 위상을 잃게 되었다.

고종이 승하한 뒤 1920년부터 일제가 선원전과 중명전 일대를 매각하여 궁역이 크게 줄어들었으며, 1933년에는 많은 건물들을 철거하고 공원으로 조성하여 일

In [92]:
# 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer in Korean.

#Context: 
{context}

#Question:
{question}

#Answer:"""
)

llm = ChatOpenAI(model_name="gpt-4.1-mini", temperature=0)

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser() # 출력에 page_content내용만 보일 수 있도록
)

In [93]:
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "인목왕후가 감금되어 있었던 곳인 어디인가?"
response = chain.invoke(question)
print(response)

인목왕후가 감금되어 있었던 곳은 석어당입니다.


In [75]:
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "고종이 침전으로 사용했던 건물이 뭐야?"
response = chain.invoke(question)
print(response)

고종이 침전으로 사용했던 건물은 석조전입니다.


In [76]:
question = "석조전에 용도가 뭐지?"
response = chain.invoke(question)
print(response)

석조전은 고종황제가 침전 겸 편전으로 사용하기 위해 1900년부터 1910년에 걸쳐 지은 서양식 석조건물입니다. 1층은 시종이 기거하는 방과 부속 시설, 2층은 대접견실과 대기실, 3층은 황제와 황후가 거처하는 침실과 여러 용도의 방으로 구성되어 있습니다.


In [79]:
question="이용시간을 알려줘"
response = chain.invoke(question)
print(response)


덕수궁의 관람시간은 09:00부터 21:00까지이며, 매일 1시간 전까지 입장해야 합니다. 매주 월요일은 휴무입니다.


In [35]:
question="덕수궁에 대해 알려줘"
response = chain.invoke(question)
print(response)

덕수궁은 원래 경운궁이라는 이름으로 불렸으며, 1611년 광해군 때부터 약 300년간 경운궁이라는 궁호가 사용되었습니다. 1897년 대한제국 출범과 함께 경운궁은 대한제국의 으뜸 궁궐이 되었고, 고종 황제가 이곳에서 황제 즉위식을 거행하였습니다. 이후 고종이 물러나면서 궁의 위상이 달라지고 이름도 덕수궁으로 바뀌었습니다.

덕수궁은 원래 매우 넓은 영역을 차지했으나, 고종이 1919년에 승하한 후 일제강점기를 거치면서 궁역이 크게 축소되고 많은 건물들이 철거되어 현재의 규모로 줄어들었습니다. 1904년 대화재로 주요 건물들이 소실되었고, 1933년에는 일본인에게 개방되기도 했습니다. 이후 2007년부터 덕수궁 복원이 시작되었습니다.

덕수궁은 임진왜란과 대한제국기의 역사를 겪은 궁궐로서 국난 극복의 상징적 공간이며, 주변의 높은 빌딩들 사이에 위치해 현대와 역사가 공존하는 장소입니다. 주요 건물로는 대한문, 중화전, 석조전 등이 있으며, 대한문은 1971년에 원래 자리에서 33m 물러난 현재 위치로 옮겨졌고 여러 차례 수리를 거쳤습니다.

덕수궁 내에는 표 파는 곳, 지하철, 화장실, 기념품점, 휠체어 보관소 등의 편의시설도 마련되어 있습니다.


In [36]:
question="롯데타워는 어디야?"
response = chain.invoke(question)
print(response)

죄송하지만, 제공된 자료에는 롯데타워의 위치에 대한 정보가 없습니다.


pdf를 불러와서 가공없이 임베딩한 결과를 저장한 vectorstore를 로드해서 편집한 md파일과 성능을 비교

In [44]:
# 저장된 데이터를 로드
dk_db = FAISS.load_local(
    folder_path="faiss_db",
    index_name="faiss_index",
    embeddings=embeddings,
    allow_dangerous_deserialization=True,
)

In [48]:
# 리트리버 생성
retriever2 = dk_db.as_retriever()

# 검색할 질의(Query)
query2 = "인목왕후가 감금되어 있었던 곳인 어디인가?"

# 리트리버를 사용하여 검색
retrieved_docs2 = retriever2.invoke(query2)
print_doc_contecnt(retrieved_docs2)

1번째 문서
내용:
# 석조전 앞의 분수

석조전 앞에 있는 정원과 청동제 분수는 서구형 정원으로 1938년에 조성된 것으로 알려져 있다. 우리의 전통 정원은 건물의 뒤에 배치하여 후원이라 하는 데 비해, 석조전은 건물 앞에 정원을 조성하였다. 또한 우리 전통 조경에서는 분수를 찾아볼 수 없는데, 이는 한국인의 자연관에서 비롯된 것이다. 우리 조상들은 물을 높은 곳에서 낮은 곳으로 흐르는 것이 자연의 이치라 여겼기에, 정원에 인공으로 폭포를 만들긴 했지만 물이 억으로 치솟는 분수는 만들지 않았던 것이다.

!Image of a fountain in front of Seokjojeon

# 덕수궁 미술관

덕수궁 미술관 圖書館

광명문 光明門

석조전 石造殿

!Illustration of Deoksugung Palace

# 석조전을 시공한 오쿠라 토목회사
------------------------------------------------------------------------------------------
2번째 문서
내용:
# 9 중명전 일원
重明殿

## 황제국 건설의 희망과 절망

중명전 일대는 경운궁을 확장할 때 가장 먼저 궁궐로 편입된 곳이지만, 경운궁과 중명전 사이에 이미 미국공사관이 자리 잡고 있던 터라 별궁처럼 되었다. 중명전은 러시아 건축가 사바틴이 설계한 2층의 붉은 벽돌 건물로 경운궁에 지어진 최초의 서양식 건물이었다. 처음에는 수옥헌으로 불리며 황실도서관으로 지어졌으나 1904년 대한제국 고종이 임시 기거하면서 연회장이나 외국 사절의 접견소로도 이용되었다. 수옥헌이 중명전으로 불리게 된 것은 1906년 이후로 보여 진다. 이 일대에는 환벽당과 만화당을 비롯한 10여 채의 전각들이 들어서 있었다. 1905년 을사늑약 체결된 곳도 중명전이고, 1907년 헤이그 만국평화회의의 특사 파견이 이루어진 곳도 중명전이다. 이처럼 중명전은 대한제국의 좌절과 국권 수호 의지가 모두 담긴 역사의 현장이다.

!Image of 중명전 일원

## 

In [49]:
# 검색할 질의(Query)
query2 = "고종이 침전으로 사용했던 건물이 뭐야?"

# 리트리버를 사용하여 검색
retrieved_docs2 = retriever2.invoke(query2)
print_doc_contecnt(retrieved_docs2)

1번째 문서
내용:
# 7 권내각사 터와 환구단
關內各司址 · 圜丘壇

----

## 환구단, 황제국 선포를 하늘에 고하다
------------------------------------------------------------------------------------------
2번째 문서
내용:
# 석조전 앞의 분수

석조전 앞에 있는 정원과 청동제 분수는 서구형 정원으로 1938년에 조성된 것으로 알려져 있다. 우리의 전통 정원은 건물의 뒤에 배치하여 후원이라 하는 데 비해, 석조전은 건물 앞에 정원을 조성하였다. 또한 우리 전통 조경에서는 분수를 찾아볼 수 없는데, 이는 한국인의 자연관에서 비롯된 것이다. 우리 조상들은 물을 높은 곳에서 낮은 곳으로 흐르는 것이 자연의 이치라 여겼기에, 정원에 인공으로 폭포를 만들긴 했지만 물이 억으로 치솟는 분수는 만들지 않았던 것이다.

!Image of a fountain in front of Seokjojeon

# 덕수궁 미술관

덕수궁 미술관 圖書館

광명문 光明門

석조전 石造殿

!Illustration of Deoksugung Palace

# 석조전을 시공한 오쿠라 토목회사
------------------------------------------------------------------------------------------
3번째 문서
내용:
1900년에 완성된 선원전전에는 숙종을 비롯한 7위의 어진(御眞, 임금의 초상화)을 모신 황실의 격을 갖추었으며, 빈전(殯殿)과 혼전(魂殿)도 설치했다. 선원전 일대는 대한제국을 상징하는 신성한 공간이었으나, 1919년 고종황제 승하 후 1년제가 끝나자마자 일제는 어진들을 돌볼 사람이 없다는 이유로 선원전을 혈의 순종이 있는 창덕궁으로 옮겼고, 그 부지는 조선은행, 식산은행, 경성일보사 등에 매각했다. 이후 해인사와 불교중앙포교소와 경성여자공립보통학교(현 덕수초등학교 터), 경기여고터가 차례로 건축되면서 이 일대는 완전히 해체되었다. 현재는 경기

In [50]:
query2 = "석조전에 용도가 뭐야?"

# 리트리버를 사용하여 검색
retrieved_docs2 = retriever2.invoke(query2)
print_doc_contecnt(retrieved_docs2)

1번째 문서
내용:
# 석조전 앞의 분수

석조전 앞에 있는 정원과 청동제 분수는 서구형 정원으로 1938년에 조성된 것으로 알려져 있다. 우리의 전통 정원은 건물의 뒤에 배치하여 후원이라 하는 데 비해, 석조전은 건물 앞에 정원을 조성하였다. 또한 우리 전통 조경에서는 분수를 찾아볼 수 없는데, 이는 한국인의 자연관에서 비롯된 것이다. 우리 조상들은 물을 높은 곳에서 낮은 곳으로 흐르는 것이 자연의 이치라 여겼기에, 정원에 인공으로 폭포를 만들긴 했지만 물이 억으로 치솟는 분수는 만들지 않았던 것이다.

!Image of a fountain in front of Seokjojeon

# 덕수궁 미술관

덕수궁 미술관 圖書館

광명문 光明門

석조전 石造殿

!Illustration of Deoksugung Palace

# 석조전을 시공한 오쿠라 토목회사
------------------------------------------------------------------------------------------
2번째 문서
내용:
1900년에 완성된 선원전전에는 숙종을 비롯한 7위의 어진(御眞, 임금의 초상화)을 모신 황실의 격을 갖추었으며, 빈전(殯殿)과 혼전(魂殿)도 설치했다. 선원전 일대는 대한제국을 상징하는 신성한 공간이었으나, 1919년 고종황제 승하 후 1년제가 끝나자마자 일제는 어진들을 돌볼 사람이 없다는 이유로 선원전을 혈의 순종이 있는 창덕궁으로 옮겼고, 그 부지는 조선은행, 식산은행, 경성일보사 등에 매각했다. 이후 해인사와 불교중앙포교소와 경성여자공립보통학교(현 덕수초등학교 터), 경기여고터가 차례로 건축되면서 이 일대는 완전히 해체되었다. 현재는 경기여고 터의 선원전 복원을 비롯한 덕수궁 장기 복원계획이 수립되어 추진 중이다.

!영친왕의 선원전 참배

13
------------------------------------------------------------------------------------------
3번째 문서
내

In [52]:
query2 = "이용시간을 알려줘"

# 리트리버를 사용하여 검색
retrieved_docs2 = retriever2.invoke(query2)
print_doc_contecnt(retrieved_docs2)

1번째 문서
내용:
### 헤이그 밀사 파견과 고종의 강제 양위

고종은 일본과 맺은 을사늑약의 부당함을 세계 각국에 알리고자 1907년 6월 네덜란드 헤이그에서 열린 제2차 만국평화회의에 특사를 파견했다. 그러나 특사들은 회의장에 들어가지도 못한 채 거절당했다. 열강의 냉담한 반응에 이준 열사는 규국하지 않고 순국하여 대한제국의 의지를 천명하였다. 이 일로 일본은 고종에 여러 처리 양해를 강요하였고, 결국 고종은 순종에게 대한제정창설을 명령케 이른다. 그러나 실제로 세자가 대신 정무를 보는 대리청정을 위임한 바였고 그 양위식은 1907년 7월 19일 중화전에서 치러졌다. 고종과 순종은 양위식 참석을 거부했다.

!중화전 구조도

### '드므'에 새겨진 '단세(檀世)'의 의미
------------------------------------------------------------------------------------------
2번째 문서
내용:
# 7 권내각사 터와 환구단
關內各司址 · 圜丘壇

----

## 환구단, 황제국 선포를 하늘에 고하다
------------------------------------------------------------------------------------------
3번째 문서
내용:
# 9 중명전 일원
重明殿

## 황제국 건설의 희망과 절망

중명전 일대는 경운궁을 확장할 때 가장 먼저 궁궐로 편입된 곳이지만, 경운궁과 중명전 사이에 이미 미국공사관이 자리 잡고 있던 터라 별궁처럼 되었다. 중명전은 러시아 건축가 사바틴이 설계한 2층의 붉은 벽돌 건물로 경운궁에 지어진 최초의 서양식 건물이었다. 처음에는 수옥헌으로 불리며 황실도서관으로 지어졌으나 1904년 대한제국 고종이 임시 기거하면서 연회장이나 외국 사절의 접견소로도 이용되었다. 수옥헌이 중명전으로 불리게 된 것은 1906년 이후로 보여 진다. 이 일대에는 환벽당과 만화당을 비롯한 10여 채의 전각들이 들어서 있었다. 1905년 을사늑약 체결된 곳

In [53]:
query2 = "덕수궁에 대해 알려줘"

# 리트리버를 사용하여 검색
retrieved_docs2 = retriever2.invoke(query2)
print_doc_contecnt(retrieved_docs2)

1번째 문서
내용:
### 헤이그 밀사 파견과 고종의 강제 양위

고종은 일본과 맺은 을사늑약의 부당함을 세계 각국에 알리고자 1907년 6월 네덜란드 헤이그에서 열린 제2차 만국평화회의에 특사를 파견했다. 그러나 특사들은 회의장에 들어가지도 못한 채 거절당했다. 열강의 냉담한 반응에 이준 열사는 규국하지 않고 순국하여 대한제국의 의지를 천명하였다. 이 일로 일본은 고종에 여러 처리 양해를 강요하였고, 결국 고종은 순종에게 대한제정창설을 명령케 이른다. 그러나 실제로 세자가 대신 정무를 보는 대리청정을 위임한 바였고 그 양위식은 1907년 7월 19일 중화전에서 치러졌다. 고종과 순종은 양위식 참석을 거부했다.

!중화전 구조도

### '드므'에 새겨진 '단세(檀世)'의 의미
------------------------------------------------------------------------------------------
2번째 문서
내용:
관람정보

# 관람정보

100-120 서울시 중구 세종대로 99 / deoksugung.go.kr / 02-771-9952

## 관람시간
09:00~21:00  
※ 매일 1시간 전까지 입장해야 하며 매주 월요일은 쉽니다

## 입장요금
- 어른(19~64세) 1,000원(10인 이상 800원)

## 무료 안내 시간

### 국어
- 화~금요일: 10:00, 11:00, 14:00, 15:00, 16:00
- 토요일: 10:00, 11:00, 14:00, 15:00, 16:30
- 일요일: 10:00, 11:00, 14:00, 15:00, 16:30

### 영어
- 화, 수, 금요일: 10:30
- 토, 일요일: 13:40

### 일어
- 화, 수, 금요일: 10:30
- 화, 수, 금요일: 14:30
- 토요일: 10:30, 14:30
- 일요일: 10:30

### 중국어
- 화, 수, 금요일: 13:40
- 토요일: 13:40
- 일요일: 13:40
------------------------

In [56]:
# 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer in Korean.

#Context: 
{context}

#Question:
{question}

#Answer:"""
)

llm = ChatOpenAI(model_name="gpt-4.1-mini", temperature=0)

chain = (
    {"context": retriever2, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser() # 출력에 page_content내용만 보일 수 있도록
)

In [57]:
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question2 = "인목왕후가 감금되어 있었던 곳인 어디인가?"
response2 = chain.invoke(question2)
print(response2)

인목왕후가 감금되어 있었던 곳에 대한 정보는 주어진 문서들에 포함되어 있지 않습니다. 따라서 답변을 드릴 수 없습니다.


In [58]:
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question2 = "고종이 침전으로 사용했던 건물이 뭐야?"
response2 = chain.invoke(question2)
print(response2)

고종이 침전으로 사용했던 건물은 선원전입니다.


In [59]:
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question2 = "석조전에 용도가 뭐지?"
response2 = chain.invoke(question2)
print(response2)

석조전은 덕수궁 내에 위치한 건물로, 서구형 건축 양식으로 지어진 궁전입니다. 주로 대한제국 시기 황실의 주요 공간으로 사용되었으며, 현재는 덕수궁 미술관 등으로 활용되고 있습니다.


In [60]:
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question2 = "이용시간을 알려줘"
response2 = chain.invoke(question2)
print(response2)

이용시간에 대한 정보는 제공된 문서 내에 없습니다.


In [61]:
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question2 = "덕수궁에 대해 알려줘"
response2 = chain.invoke(question2)
print(response2)

덕수궁은 1897년 대한제국의 으뜸 궁궐로 지정된 경운궁을 중심으로 형성된 궁궐입니다. 원래 경운궁은 예원학교, 덕수초등학교, 옛 경기여고 터를 포함하는 넓은 영역이었으나, 1919년 고종황제 서거 이후 궁역이 해체되면서 현재의 덕수궁 영역으로 축소되었습니다.

덕수궁 내에는 서양식 건축물인 석조전이 있으며, 석조전 앞에는 1938년에 조성된 서구형 정원의 청동제 분수가 있습니다. 이는 전통 한국 정원과 달리 건물 앞에 조성된 점과 분수의 형태가 독특한 특징입니다. 전통적으로 한국 정원은 건물 뒤에 후원으로 조성되며, 물이 높은 곳에서 낮은 곳으로 흐르는 자연의 이치를 중시해 인공 분수는 없었습니다.

또한, 덕수궁에는 돈덕전, 중명전, 환벽정, 만희당 등 여러 건물이 있었으나 일제강점기와 경술국치 이후 일부가 철거되거나 해체되었습니다. 덕수궁은 대한제국의 역사적 현장이자 고종과 순종의 중요한 무대였던 곳입니다.

덕수궁 관람은 서울시 중구 세종대로 99에 위치해 있으며, 관람시간은 오전 9시부터 오후 9시까지입니다(매주 월요일 휴무). 입장료는 어른 기준 1,000원이며, 무료 안내 시간에 맞춰 국어, 영어, 일어, 중국어 해설도 제공됩니다.


In [62]:
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question2 = "롯데타워는 어디야?"
response2 = chain.invoke(question2)
print(response2)

롯데타워에 대한 정보는 제공된 문서들에 포함되어 있지 않습니다. 따라서 롯데타워가 어디에 있는지 알 수 없습니다.


In [75]:
# 저장된 데이터를 로드
dk_db2 = FAISS.load_local(
    folder_path="faiss_db2",
    index_name="faiss_index",
    embeddings=embeddings,
    allow_dangerous_deserialization=True,
)

In [77]:
# 리트리버 생성
retriever3 = dk_db2.as_retriever()

# 검색할 질의(Query)
query3 = "인목왕후가 감금되어 있었던 곳인 어디인가?"

# 리트리버를 사용하여 검색
retrieved_docs3 = retriever3.invoke(query3)
print_doc_contecnt(retrieved_docs3)

1번째 문서
내용:
석어당은 선조의 계비인 인목왕후 김씨가 10여 년간 감금생활을 했던 곳이다. 광해군은 왕위에 오른 후 자신의 왕위를 위협할 가능성이 있는 형제들을 차례로 제거하고 인목대비를 폐위시켜 경운궁에 유폐했다. 이때 경운궁은 서궁(西宮)으로 불렸다. 서궁 유폐는 결국 반정을 일으키는 구실이 되었다. 반정이 성공한 능양군(인조)은 경운궁으로 입궐해를 찾아 정통성을 인정받고 여기서 즉위했다. 유폐의 한이 맺혀 있던 인목대비는 석어당 앞마당에 광해군을 끌어내리고 36조의 죄를 물은 후 능운궁에서 묘사를 전하였다.
  
  
  
4 함녕전과 덕홍전

# 함녕전과 덕홍전
### 咸寧殿 · 德弘殿

### 황제의 침전과 접견실
------------------------------------------------------------------------------------------
2번째 문서
내용:
## 환구단, 황제국 선포를 하늘에 고하다

현재의 덕수궁 동쪽 담장 안쪽과 담장 너머 서울광장 일부는 궁궐 안 관청들이 있던 권내각사 일원이었다. 경운궁으로 정궁을 이전하면서 군사권을 관장하던 원수부와 황실의 업무를 보던 궁내부를 비롯해 기양원, 태의원, 전화국 등 여러 관청이 권내에 위치하게 되었다. 후에 태평로 개설로 인해 절반 이상의 건각이 사라졌고, 나머지 건각 역시 1933년 공원으로 조성되는 과정에서 철거되었다. 환구단은 현 서울광장 너머에 있었던 남별궁 터(현 조선호텔 자리)에 건축되었다. 남별궁은 본디 태종의 둘째 공주인 경정공주가 거주했던 곳으로 1583년(선조 16년)에 의안군이 거주하면서 남별궁이라 불렸다. 이후 인조 때 중국 사신을 접대하던 태평관이 철폐되자 중국 사신의 거처로 사용되다가, 1897년 고종이 황제에 오름을 하늘에 고하기 위한 환구단이 이곳에 건축되었다. 이후 1914년 조선철도호텔 건설 때 파괴되었으나 신위를 모셨던 황궁우와 돌북(石鼓)이 남아 대한제국의 흔적을 보여주고 있다. 환구단은 사적 제157호로 지정되었다.
